In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("npr.csv")

In [3]:
df.head(10)

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."
5,I did not want to join yoga class. I hated tho...
6,With a who has publicly supported the debunk...
7,"I was standing by the airport exit, debating w..."
8,"If movies were trying to be more realistic, pe..."
9,"Eighteen years ago, on New Year’s Eve, David F..."


In [4]:
len(df['Article'])

11992

In [5]:
df.shape

(11992, 1)

### Preprocessing 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
# we are going to build a vocabulary by ignoring few words that has high document frequencies.
# max_df(maximum doc frequencies) helps to get rid of most commonly used words in the document (value between 0 and 1).

cv = CountVectorizer(max_df = 0.99 ,min_df = 2, stop_words = 'english')

In [8]:
doc_term_matrix = cv.fit_transform(df['Article'])

In [9]:
doc_term_matrix

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
LDA = LatentDirichletAllocation(n_components = 7,random_state=42)

In [12]:
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=7, random_state=42)

### Grab the Vocabulary of words 

In [13]:
len(cv.get_feature_names())

54777

In [14]:
type(cv.get_feature_names())

list

In [15]:
# diplay the words in the vocabulary randomly

import random

random_word = random.randint(0,54777)

cv.get_feature_names()[random_word]

'retested'

### Grab the topics

In [16]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [17]:
len(LDA.components_)

7

In [18]:
LDA.components_.shape #Topics,words

(7, 54777)

In [19]:
single_topic = LDA.components_[0]

In [20]:
# argsort() -- sorts the array in ascending order and gives the index values
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993], dtype=int64)

In [21]:
# TOP 10 values (10 greatest values)
 
single_topic.argsort()[-10:] # returns the last 10 values of .argsort()

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993], dtype=int64)

In [22]:
top_20_words = single_topic.argsort()[-20:]

In [23]:
for i in top_20_words:
    print(cv.get_feature_names()[i])

president
state
tax
insurance
trump
companies
money
year
federal
000
new
percent
government
company
million
care
people
health
said
says


* The words represent either politics or insurance

### Grab the highest probability words 

In [24]:
for index,topic in enumerate(LDA.components_):
    print(f"The top 15 words in the topic# {index+1}")
    print([cv.get_feature_names()[index] for index in topic.argsort()[-15:]])
    print("\n")

The top 15 words in the topic# 1
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


The top 15 words in the topic# 2
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


The top 15 words in the topic# 3
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


The top 15 words in the topic# 4
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


The top 15 words in the topic# 5
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


The top 15 words in the topic# 6
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time'

In [25]:
doc_term_matrix

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [26]:
# Create actual topics for the document term matrix

topics = LDA.transform(doc_term_matrix)

In [27]:
topics[0].argmax()

1

In [28]:
df['Topic'] = topics.argmax(axis=1)

In [29]:
df

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
...,...,...
11987,The number of law enforcement officers shot an...,1
11988,"Trump is busy these days with victory tours,...",4
11989,It’s always interesting for the Goats and Soda...,3
11990,The election of Donald Trump was a surprise to...,4
